In [12]:
# whether to log each feature and sequence status
verbose = True

In [13]:
import gc
import os
import pandas as pd
pd.options.display.max_rows = 5000
import numpy as np
import json
import datetime
import matplotlib.pyplot as plt
import itertools
import sys
sys.path.append('..')

In [14]:
# setup paths
pwd = os.getcwd().replace("notebooks","")
path_cache = pwd + 'cache/'
path_data = pwd + 'data/'

In [41]:
from deepvideoclassification.architectures import Architecture

In [15]:
# setup logging
# any explicit log messages or uncaught errors to stdout and file /logs.log
import logging
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(threadName)-12.12s] [%(levelname)-5.5s]  %(message)s",
    handlers=[
        logging.FileHandler("{0}/{1}.log".format(pwd, "logs")),
        logging.StreamHandler()
    ])
# init logger
logger = logging.getLogger()
# make logger aware of any uncaught exceptions
def handle_exception(exc_type, exc_value, exc_traceback):
    if issubclass(exc_type, KeyboardInterrupt):
        sys.__excepthook__(exc_type, exc_value, exc_traceback)
        return

    logger.error("Uncaught exception", exc_info=(exc_type, exc_value, exc_traceback))
sys.excepthook = handle_exception

# Create list of experiments to run

In [20]:
# parameters to grid search over
pooling = 'max'
layer_sizes = [512, 256, 128, 0]
dropouts = [0.2]
sequence_lengths = [3,5,10]
sequence_model_layer_counts = [1,2]
sequence_models = ["LSTM", "SimpleRNN", "GRU", "Convolution1D"]
pretrained_model_names = ['inception_resnet_v2', 'inception_v3', 'mobilenetv2_1.00_224', 'resnet50', 'vgg16', 'xception']

In [21]:
# init model id - need to make sure we pick up where we leave off don't overwrite it between batches
model_id = 0

# init list of experiments
experiments = []

In [22]:
####################
### image_MLP_frozen 
####################

for pretrained_model_name in pretrained_model_names:
    for layer_1_size in layer_sizes:
        for layer_2_size in layer_sizes:
            for layer_3_size in layer_sizes:
                for dropout in dropouts:

                    # build experiment parameters
                    experiment = {}
                    
                    experiment['model_id'] = model_id
                    experiment['architecture'] = 'image_MLP_frozen'
                    experiment['sequence_length'] = 1
                    experiment['pretrained_model_name'] = pretrained_model_name
                    experiment['layer_1_size'] = layer_1_size
                    experiment['layer_2_size'] = layer_2_size
                    experiment['layer_3_size'] = layer_3_size
                    experiment['dropout'] = dropout
                    experiment['pooling'] = 'max' # outperforms avg across all parameters
                    
                    # add to list of experiments
                    experiments.append(experiment)
                    
                    model_id+=1

In [23]:
####################
### video_MLP_concat
####################

for sequence_length in sequence_lengths:
    for pretrained_model_name in pretrained_model_names_bucketed:
        for layer_1_size in layer_sizes:
            for layer_2_size in layer_sizes:
                for layer_3_size in layer_sizes:
                    for dropout in dropouts:

                        # build experiment parameters
                        experiment = {}
                        
                        experiment['model_id'] = model_id
                        experiment['architecture'] = 'video_MLP_concat'
                        experiment['pretrained_model_name'] = pretrained_model_name
                        experiment['layer_1_size'] = layer_1_size
                        experiment['layer_2_size'] = layer_2_size
                        experiment['layer_3_size'] = layer_3_size
                        experiment['dropout'] = dropout
                        experiment['pooling'] = 'max' # outperforms avg across all parameters
                        experiment['sequence_length'] = sequence_length

                        # add to list of experiments
                        experiments.append(experiment)
                        model_id+=1

In [24]:
######################
### video_LRCNN_frozen
######################

for sequence_length in sequence_lengths:
    for pretrained_model_name in pretrained_model_names_bucketed:
        for layer_1_size in layer_sizes:
            for layer_2_size in layer_sizes:
                for layer_3_size in layer_sizes:
                    for dropout in dropouts:
                        for sequence_model in sequence_models:
                            for sequence_model_layers in sequence_model_layer_counts:

                                # build experiment parameters
                                experiment = {}

                                experiment['model_id'] = model_id
                                experiment['architecture'] = 'video_LRCNN_frozen'
                                experiment['pretrained_model_name'] = pretrained_model_name
                                experiment['layer_1_size'] = layer_1_size
                                experiment['layer_2_size'] = layer_2_size
                                experiment['layer_3_size'] = layer_3_size
                                experiment['dropout'] = dropout
                                experiment['pooling'] = 'max' # outperforms avg across all parameters
                                experiment['sequence_model'] = sequence_model
                                experiment['sequence_model_layers'] = sequence_model_layers
                                experiment['sequence_length'] = sequence_length

                                # add to list of experiments
                                experiments.append(experiment)
                                model_id+=1

In [25]:
########################
### convert to dataframe
########################

experiments = pd.DataFrame(experiments)

In [27]:
############################################
### remove invalid experiment configurations
############################################

# Just won't run experiments for those model_ids - not an error that model ids not congituous count from 0!

# delete video experiments with 0 neurons in a layer with nonzero neurons in later layers
experiments = experiments[~((experiments['layer_1_size'] == 0) & (experiments['layer_2_size'] > 0))]
experiments = experiments[~((experiments['layer_1_size'] == 0) & (experiments['layer_3_size'] > 0))]
experiments = experiments[~((experiments['layer_2_size'] == 0) & (experiments['layer_3_size'] > 0))]

# delete video experiments where convolution_kernel_size > sequence_length (convolution_kernel_size defaults to 3 and not set in this batch)
experiments = experiments[~((experiments['sequence_model'] == 'Convolution1D') & (experiments['sequence_length']<=3))]

In [28]:
# delete LRCNN_frozen experiments with layer_1_size == 0
experiments = experiments[~((experiments['architecture'] == 'video_LRCNN_frozen') & (experiments['layer_1_size']==0))]

In [30]:
##################################
### output experiment batch to CSV
##################################
experiments.to_csv(pwd +  'experiments.csv', index=False)

In [31]:
print(experiments.shape)
experiments.tail().T

(3174, 11)


,5435,5436,5437,5438,5439
architecture,video_LRCNN_frozen,video_LRCNN_frozen,video_LRCNN_frozen,video_LRCNN_frozen,video_LRCNN_frozen
dropout,0.2,0.2,0.2,0.2,0.2
layer_1_size,128,128,128,128,128
layer_2_size,0,0,0,0,0
layer_3_size,0,0,0,0,0
model_id,5435,5436,5437,5438,5439
pooling,max,max,max,max,max
pretrained_model_name,resnet50,resnet50,resnet50,resnet50,resnet50
sequence_length,10,10,10,10,10
sequence_model,SimpleRNN,GRU,GRU,Convolution1D,Convolution1D


In [39]:
exps = experiments.to_dict(orient="index").values()

# Run grid search

In [43]:
for experiment in exps:
    print(str(experiment["model_id"]) + "   " + "X"*60)
    print(experiment)
    
    # delete existing results
    if os.path.exists(pwd + 'models/' + str(experiment["model_id"]) + '/results.json'):
        rmtree(pwd + 'models/' + str(experiment["model_id"]) + '/')

    architecture = Architecture(model_id = experiment['model_id'], 
                                architecture = experiment['architecture'], 
                                sequence_length = experiment['sequence_length'], 
                                pretrained_model_name = experiment['pretrained_model_name'],
                                pooling = experiment['pooling'],
                                sequence_model = experiment['sequence_model'],
                                sequence_model_layers = experiment['sequence_model_layers'],
                                layer_1_size = experiment['layer_1_size'],
                                layer_2_size = experiment['layer_2_size'],
                                layer_3_size = experiment['layer_3_size'],
                                dropout = experiment['dropout'],
                                verbose=True)

    architecture.train_model()

    gc.collect()